# Hotspot Transit Improvements

Criteria: 
* transit routes that do not run on the SHN
* average route speed is above **12 mph**  **AND**
* 50%+ of its trips are competitive against a car traveling the same path (competitive is 1.5x...so bus can take 50% more time)

In [ ]:
import branca 
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import Markdown

from shared_utils import calitp_color_palette as cp
from shared_utils import geography_utils

catalog = intake.open_catalog("./*.yml")

CUSTOM_CATEGORICAL = [
    cp.CALITP_CATEGORY_BRIGHT_COLORS[2], # yellow
    cp.CALITP_CATEGORY_BRIGHT_COLORS[1], # orange
    cp.CALITP_CATEGORY_BRIGHT_COLORS[0], # blue
    cp.CALITP_CATEGORY_BRIGHT_COLORS[5], # purple  
    cp.CALITP_CATEGORY_BRIGHT_COLORS[3], # green
]

ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=35)

In [ ]:
gdf = catalog.bus_routes_all_aggregated_stats.read().to_crs(geography_utils.WGS84)

include = ["intersects_shn", "other"]
gdf = gdf[gdf.category.isin(include)].reset_index(drop=True)

In [ ]:
METRIC_CUTOFFS = {
    # show routes with average speeds >= 12 mph
    "mean_speed_mph": 12, 
    # show routes with >= 10% of trips being competitive against a car 
    # competitive is if it takes bus no longer than +20% longer than a car
    "pct_trips_competitive": 0.50,
}

In [ ]:
def subset_to_district_sort_metric(gdf: gpd.GeoDataFrame, 
                                   district: str,
                                   speed_cutoff: int = 12,
                                   pct_competitive_cutoff: float = 0.05
                                  ): 
    keep_cols = ["calitp_itp_id", "route_id", 
            "mean_speed_mph", "pct_trips_competitive",
            "category", "geometry"]
    
    df = (gdf[(gdf.caltrans_district == district) & 
              (gdf.mean_speed_mph >= speed_cutoff) & 
              (gdf.pct_trips_competitive >= pct_competitive_cutoff)
             ]
          .reset_index(drop=True)
          [keep_cols]
         )
    
    df2 = df.assign(
        mean_speed_mph = df.mean_speed_mph.round(2),
        pct_trips_competitive = df.pct_trips_competitive.round(3)
    )
               
    return df2

In [ ]:
TILES = "CartoDB Positron" 
def make_map(gdf: gpd.GeoDataFrame, district: str):        
    m = gdf.explore("mean_speed_mph", categorical = False, 
                    cmap = ZERO_THIRTY_COLORSCALE, tiles = TILES)
    
    display(m)
    
    m2 = gdf.explore("route_id", categorical = True, 
                    cmap = "tab20", tiles = TILES)
    
    display(m2)
    
    display(gdf.sort_values(
        ["mean_speed_mph", "pct_trips_competitive"], 
        ascending=[True, True]).drop(columns = ["geometry", "category"]))

In [ ]:
districts = gdf[gdf.caltrans_district.notna()].caltrans_district.unique().tolist()

for d in sorted(districts):
    display(Markdown(f"### District {d}"))

    subset = subset_to_district_sort_metric(
        gdf, d, 
        speed_cutoff = METRIC_CUTOFFS["mean_speed_mph"],
        pct_competitive_cutoff = METRIC_CUTOFFS["pct_trips_competitive"]
    )

    if len(subset) > 0:
        make_map(subset, d)
    else:
        print("No routes meet this criteria")